In [ ]:
import re

from sklearn import datasets
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

#### TF-IDF
$TF$ - частота встречания терма (term frequency)<br>
$IDF$ - обратная частота встречания терма в документах (invert document frequency)

$$tf_{i, j} = \frac{N_i}{\sum\limits_j N_j}$$<br>
(1) $$idf_{i} = log(\frac{|D|}{|\{d_k \in D | i \in d_k \}|}) \approx log (\frac{|D|}{df(i) + 1}) \approx log (\frac{|D| + 1}{df(i) + 1}) + 1$$

(2) $$idf_{i} = log(\frac{|D|}{|\{d_k \in D | i \in d_k \}|}) + 1$$


### Это ознакомительное задание. В нём от вас требуется
    
* Задать правильную формулу TF.IDF в функции `__calc_token_weight`,
* Передать в неё правильные параметры.

_NB! В этой задаче в словарях с частотами бывает удобно использовать_ `defaultdict` _или_ `Counter`

In [ ]:
from collections import defaultdict, deque, Counter

In [ ]:
import math

In [ ]:
class TfIdfExtractor:
    def __init__(self, ngrams=1):
        self.token_pattern = re.compile(r"(?u)\b\w\w+\b")
        self.ngrams = ngrams

        self.docs_with_token = None
        self.token_hits_count_in_doc = None
        self.token_count_in_doc = None
        self.docs_count = None

    def compute_counts_in_collection_collection(self, documents):
        """
        Заполняем словари (хэшмэпы, ассоциативные массивы) разных частот для последующего вычисления tf-idf
        :param documents: сырые тексты
        """
        self.docs_with_token, self.token_hits_count_in_doc, self.token_count_in_doc = self.__count_tokens(documents)
        self.docs_count = len(documents)

    def __count_tokens(self, documents):
        """
        Реализуйте метод, который строит словари (хэшмэпы, ассоциативные массивы) разных частот для последующего вычисления tf-idf.
        Здесь не требуется никаких трюков, тренируемся считать явно.

        :param documents: сырые тексты
        :return: кортеж-тройка:
        docs_with_token,            # словарь: docs_with_token[токен] = число содержащих его документов
        token_hits_count_in_doc,    # словарь: token_hits_count_in_doc[документ][токен] = сколько раз токен встратился в данном документе
        token_count_in_doc          # словарь: token_count_in_doc[документ] = сколько всего токенов в документе
        """

        docs_with_token = defaultdict(int)
        token_hits_count_in_doc = []
        token_count_in_doc = []

        for doc in documents:

            # частоты токенов в этом документе
            token_dictionary = defaultdict(int)
            token_count = 0

            for token in self.__get_ngrams(doc):
                #token = token.lower()
                # считаем частоты отдельных токенов
                if token_dictionary[token] == 0:
                  token_count += 1
                token_dictionary[token] += 1


            token_hits_count_in_doc.append(token_dictionary)
            token_count_in_doc.append(token_count)
            # для каждого токена ведём учёт числа документов, в которых он встретился
            for token in token_dictionary.keys():
              docs_with_token[token] += 1
              

        return docs_with_token, token_hits_count_in_doc, token_count_in_doc

    def __get_ngrams(self, doc):
        """
        Функция, вычисляющая tf-idf, для которого всё подготовлено.
        При необходимости используйте функцию findall() у поля self.token_pattern и английские стоп-слова.

        :param doc: документ
        :return: tokens список, состоящий из n-грам
        """
        tokens = []
        clear_doc = [word for word in self.token_pattern.findall(doc)
                    if word not in ENGLISH_STOP_WORDS]

        window = deque()
        for word in clear_doc:
          word = word.lower()
          if len(window) == self.ngrams:
            tokens.append("_".join(window))
            window.popleft()
          window.append(word)

        return tokens

    def __calc_token_weight(self, document_count, docs_with_token, tokens_in_doc, tokens_total):
        """
        Метод, вычисляющий tf-idf, для которого всё подготовлено.
        При необходимости используйте math.log.

        :param document_count: общее число документов
        :param docs_with_token: число документов
        :param tokens_in_doc: сколько раз токен встретился в документе
        :param tokens_total: общее число токенов в данном документе
        :return: tf.idf
        """

        idf = math.log(document_count / docs_with_token) + 1
        tf = tokens_in_doc / tokens_total
        return tf * idf

    def extract_tfidf(self, topn_docs):
        """
        Считаем топ по tf-idf для первых N документов.

        :param topn_docs: число документов из списка
        :return: extracted_tfidf список пар <токен, tf-idf>, отсортированных по найденным tf-idf
        """

        extracted_tfidf = []
        for n_doc in range(topn_docs):
            token_weights = {}

            for token in self.token_hits_count_in_doc[n_doc]:
                token_weights[token] = self.__calc_token_weight(self.docs_count,
                                                          # число документов, содержащих token
                                                          self.docs_with_token[token],
                                                          # сколько раз встретился токен token в документе n_doc
                                                          self.token_hits_count_in_doc[n_doc][token],
                                                          # сколько токенов в документе n_doc
                                                          self.token_count_in_doc[n_doc])

            # сортируем по tf-idf
            desc_tfidf_tokens = sorted(token_weights.items(), key=lambda x: (x[1], x[0]), reverse=True)
            extracted_tfidf.append(desc_tfidf_tokens)

        return extracted_tfidf

Загрузим наш первый датасет

In [ ]:
# "истинные" наиболее представительные слова в первых 10 документах
reference = [
    "cheaper supply orbits reach economies c5hcbo alike allen ground repairstation",
    "reston overhead wrap allen leadership fee integration dennis centers nasa",
    "revolt grasp alaska files acad3 prograsm geta cshow autocad 124722",
    "list rankings krumenaker 71160 2356 source larry compuserve traffic unsubscribed",
    "class foundation banquet teachers dinner teaching studies space lichtenberg planetary",
    "access muc hicksville flaking expecting redundancy navstar bird pat digex",
    "lehigh children abominable tfv0 ucdavis wealth starving capital games dan",
    "processors silicon lower slower ssrt higher access future hjistorical germanium",
    "wpi maverick giaquinto worcester novice 2280 01609 information shuttles periodicals",
    "accelerations henry toronto acceleration andrew immersion generalizes endured efpx7ws00uh7qaop1s zoology"
]

In [ ]:
CHECKED_DOCS = len(reference)
CHECKED_TOP = len(reference[0].split(" "))

In [ ]:
CHECKED_DOCS, CHECKED_TOP

(10, 10)

In [ ]:
# один из стандартных наборов данных для классификации текстов
newsgroups = datasets.fetch_20newsgroups(subset='all', categories=['sci.space'])

# тексты
documents = newsgroups.data

Запустим подсчёт TFIDF сначала на униграммах

In [ ]:
extractor = TfIdfExtractor(ngrams=1)
extractor.compute_counts_in_collection_collection(documents)

In [ ]:
precision_accumulator = 0.0
extracted_tfidf = extractor.extract_tfidf(CHECKED_DOCS)

# посчитаем топ по tf-idf для первых десяти документов
for n_doc in range(CHECKED_DOCS):
    # сортируем по tf-idf
    desc_tfidf_tokens = extracted_tfidf[n_doc]
    tfidf_top = list(map(lambda x: x[0], desc_tfidf_tokens))[:CHECKED_TOP]

    # можно распечатать и посмотреть, отличается ли порядок (не должен)
    print(" ".join(tfidf_top))
    print(reference[n_doc])

    # вычисляем что-то вроде Precision@10
    tfidf_top_set = set(tfidf_top)
    in_reference = len(tfidf_top_set.intersection(set(reference[n_doc].split(" "))))
    precision = in_reference / CHECKED_TOP

    print(n_doc, "Precision: %.1f%%" % (precision * 100.0))
    precision_accumulator += precision

# необходимое, но не достаточное условие: у правильного решения здесь должно быть ровно 100%
print("Avg precision: %.2f%%" % (float(precision_accumulator) / CHECKED_DOCS * 100.0))

cheaper supply orbits reach economies c5hcbo alike allen ground repairstation
cheaper supply orbits reach economies c5hcbo alike allen ground repairstation
0 Precision: 100.0%
reston overhead wrap allen leadership fee integration dennis centers nasa
reston overhead wrap allen leadership fee integration dennis centers nasa
1 Precision: 100.0%
revolt grasp alaska files acad3 prograsm geta cshow autocad 124722
revolt grasp alaska files acad3 prograsm geta cshow autocad 124722
2 Precision: 100.0%
list rankings krumenaker 71160 2356 source larry compuserve traffic unsubscribed
list rankings krumenaker 71160 2356 source larry compuserve traffic unsubscribed
3 Precision: 100.0%
class foundation banquet teachers dinner teaching studies space lichtenberg planetary
class foundation banquet teachers dinner teaching studies space lichtenberg planetary
4 Precision: 100.0%
access muc hicksville flaking expecting redundancy navstar bird digex positions
access muc hicksville flaking expecting redundan

Теперь прогоним наш подсчёт на биграммах

In [ ]:
# "истинные" наиболее представительные словосочетания в первых 10 документах
reference = [
    'satellite_orbit supply_make supply_facility sources_cheaper source_supply scale_allen repair_supply remember_presence ready_source reached_ready',
    'large_scale wrap_overhead work_it stated_wrap lack_leadership wrong_nasa successful_large single_important scale_effort reston_nasa',
    'alaska_edu acad3_alaska think_cshow space_design slide_shows shuttle_design shows_think revolt_look revolt_if related_items',
    'larry_krumenaker 71160_2356 2356_compuserve compuserve_com won_answer unsubscribed_list unfortunately_did traffic_rankings traffic_given temporarily_unsubscribed',
    'studies_foundation planetary_studies dinner_banquet the_class teaching_newest space_teaching newest_frontier hours_graduate graduate_credit foundation_the',
    'users_areas soon_orbit redundancy_plane provide_redundancy orbit_major orbit_hicksville net_bird needed_provide muc_user major_users',
    'lehigh_edu ucdavis_edu workers_end wo_man wealth_way way_ist viewpoint_subject tv_companies trying_everyone thats_abominable',
    'computer_technology what_argument weight_ssrt times_lower throw_hjistorical this_kind think_dc theoretically_future technology_stated technology_actually',
    'wpi_wpi wpi_edu maverick_wpi space_program worcester_ma todd_giaquinto suggest_books subject_general sites_novice shuttles_history',
    'andrew_cmu water_immersion violent_deceleration using_water think_30 talking_sustained sustained_acceleration still_higher sounds_bit odd_gees'
]

In [ ]:
bigram_extractor = TfIdfExtractor(ngrams=2)
bigram_extractor.compute_counts_in_collection_collection(documents)

In [ ]:
precision_accumulator = 0.0
extracted_tfidf = bigram_extractor.extract_tfidf(CHECKED_DOCS)

for n_doc in range(CHECKED_DOCS):
    desc_tfidf_tokens = extracted_tfidf[n_doc]
    tfidf_top = list(map(lambda x: x[0], desc_tfidf_tokens))[:CHECKED_TOP]

    tfidf_top_set = set(tfidf_top)
    in_reference = len(tfidf_top_set.intersection(set(reference[n_doc].split(" "))))
    precision = in_reference / CHECKED_TOP

    print(n_doc, "Precision: %.1f%%" % (precision * 100.0))
    precision_accumulator += precision

print("Avg precision: %.2f%%" % (float(precision_accumulator) / CHECKED_DOCS * 100.0))

0 Precision: 100.0%
1 Precision: 100.0%
2 Precision: 100.0%
3 Precision: 100.0%
4 Precision: 100.0%
5 Precision: 100.0%
6 Precision: 100.0%
7 Precision: 100.0%
8 Precision: 100.0%
9 Precision: 100.0%
Avg precision: 100.00%


А теперь попробуйте посчитать tf-idf на любом значении параметра `ngrams` 

In [ ]:
ngram_extractor = TfIdfExtractor(ngrams=3)
ngram_extractor.compute_counts_in_collection_collection(documents)

In [ ]:
precision_accumulator = 0.0
extracted_tfidf = ngram_extractor.extract_tfidf(CHECKED_DOCS)

for n_doc in range(CHECKED_DOCS):
    desc_tfidf_tokens = extracted_tfidf[n_doc]
    tfidf_top = list(map(lambda x: x[0], desc_tfidf_tokens))[:CHECKED_TOP]
    print(f'TOP-{CHECKED_TOP} important words for {n_doc}th document: {tfidf_top}')

TOP-10 important words for 0th document: ['writes_the_biggest', 'value_space_if', 'the_biggest_problem', 'supply_make_cheaper', 'supply_facility_adds', 'space_if_satellite', 'space_based_sources', 'sources_cheaper_reach', 'source_supply_make', 'scale_allen_lady']
TOP-10 important words for 1th document: ['successful_large_scale', 'single_important_successful', 'large_scale_effort', 'important_successful_large', 'charge_fee_you', 'wingo_cspara_decnet', 'judy_uh_edu', 'fedex_msfc_nasa', 'decnet_fedex_msfc', 'cspara_decnet_fedex']
TOP-10 important words for 2th document: ['acad3_alaska_edu', 'think_cshow_liek', 'subject_space_design', 'station_designs_finished', 'space_shuttle_design', 'space_related_items', 'space_design_movies', 'slide_shows_think', 'shuttle_design_autocad', 'shows_think_cshow']
TOP-10 important words for 3th document: ['71160_2356_compuserve', '2356_compuserve_com', 'won_answer_larry', 'unsubscribed_list_cut', 'unfortunately_did_clip', 'traffic_rankings_listserv', 'tra